In [10]:
import yfinance as yf
import numpy as np
import pandas as pd
from keras.models import load_model

# Load the saved model
model = load_model('lstm_model')

# Ask for the stock symbol
stock = input('Enter the stock symbol: ')

stock = yf.Ticker(stock)

# Fetch the last 30 days of stock data
hist = stock.history(period="1mo")

hist

Enter the stock symbol:  MSFT


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2023-03-14 00:00:00-04:00,256.750000,261.070007,255.860001,260.790009,33620300,0.0,0.0
2023-03-15 00:00:00-04:00,259.980011,266.480011,259.209991,265.440002,46028000,0.0,0.0
2023-03-16 00:00:00-04:00,265.209991,276.559998,263.279999,276.200012,54768800,0.0,0.0
2023-03-17 00:00:00-04:00,278.260010,283.329987,276.320007,279.429993,69527400,0.0,0.0
2023-03-20 00:00:00-04:00,276.980011,277.480011,269.850006,272.230011,43466600,0.0,0.0
2023-03-21 00:00:00-04:00,274.880005,275.000000,269.519989,273.779999,34558700,0.0,0.0
2023-03-22 00:00:00-04:00,273.399994,281.040009,272.179993,272.290009,34873300,0.0,0.0
2023-03-23 00:00:00-04:00,277.940002,281.059998,275.200012,277.660004,36610900,0.0,0.0
2023-03-24 00:00:00-04:00,277.239990,280.630005,275.279999,280.570007,28172000,0.0,0.0


In [ ]:
# Check if the DataFrame is empty
if hist.empty:
    print('Error: No data available for the stock symbol', stock)
else:
    # Check if the DataFrame has at least 30 rows
    if len(hist) < 31:
        print('Error: Not enough historical data available for', stock)
    else:
        # Calculate the average closing price for the last 30 days
        avg_price = np.mean(hist['Close'])

        # Normalize the data using Min-Max scaling
        max_price = np.max(hist['Close'])
        min_price = np.min(hist['Close'])
        df_norm = (hist - min_price) / (max_price - min_price)

        # Reshape the data to match the model's input shape
        X = np.array(df_norm[-30:]).reshape((7, 60, 1))

        # Make predictions using the loaded model
        y_pred = model.predict(X)

        # Denormalize the predicted output
        y_pred_denorm = y_pred * (max_price - min_price) + min_price

        # Print the possible next day's average closing price
        print('Possible next day\'s average closing price: ', y_pred_denorm[0][0])

